In [ ]:
import requests
import json
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
WIT_API_HOST = 'https://api.wit.ai'
WIT_API_VERSION = '20200513'

In [ ]:
with open('token.txt') as file:
    TOKEN = file.read()
    
HEADERS = {
    'authorization': 'Bearer ' + TOKEN,
    'accept': 'application/vnd.wit.' + WIT_API_VERSION + '+json'
}

In [ ]:
def getQueries():
    rsp = requests.get(WIT_API_HOST + '/utterances', headers=HEADERS, params={'limit': 1000})
    return rsp.json()

In [ ]:
def delQueries(queries):
    if not queries:
        print('empty list, aborting')
        return
    confirm = input('CONFIRM DELETION: ')
    if confirm != 'yes':
        print('aborting delete')
        return
    print('deleting the following. \n')
    print(queries)
    
    queriesDelete = [{k: v for k, v in query.items() if k.startswith('text')} for query in queries]
    rsp = requests.delete(WIT_API_HOST + '/utterances', headers=HEADERS, data=str(queriesDelete))
    
    print(rsp.json())

In [ ]:
# [a,b,c]
def getExamples(filename):
    examples = np.loadtxt(filename, dtype='str', delimiter='\n')
    return examples

In [ ]:
# [[a,x],[b,x],[c,x]]
def labelExamples(examples, label):
    reshaped = np.reshape(examples, (-1, 1))
    labels = np.full(examples.shape[0], label).reshape(-1, 1)
    return np.concatenate((reshaped, labels), 1)

In [193]:
# [ task1 [[a,x],[b,x],[c,x]] , task2 [[d,y],[e,y],[f,y]] ]
def getLabeledExamples(filenames, labels):
    data = []
    for i in range(0, len(filenames)):
        labeled = labelExamples(getExamples(filenames[i]), labels[i])
        data.append(labeled)
    return data

In [216]:
# [ train: task1 + task2 , test: task1 + task2 ]
def formTrainTest(data):
    trains = []
    tests = []
    for dataset in data:
        a, b = train_test_split(dataset, test_size=0.5)
        trains.append(a)
        tests.append(b)
        
    train = np.empty((0,2))
    test = np.empty((0,2))
    for i in range(len(trains)):
        train = np.concatenate((train, trains[i]))
        test = np.concatenate((test, tests[i]))
        
    return train, test

In [233]:
def examplesToJson(examples):
    json = []
    for example in examples:
        json.append({'text': example[0], 'intent': example[1]})
    return json

In [234]:
data = getLabeledExamples(['class.txt', 'reg.txt'], ['task_class', 'task_reg'])
train, test = formTrainTest(data)
trainJson = examplesToJson(train)
testJson = examplesToJson(test)

In [240]:
str(trainJson)

"[{'text': 'decide if an email was spoofed', 'intent': 'task_class'}, {'text': 'create something that predicts if a crime will occur', 'intent': 'task_class'}, {'text': 'categorize car pics between sedan or minivan', 'intent': 'task_class'}, {'text': 'phishing detection', 'intent': 'task_class'}, {'text': 'classify between different types of groceries', 'intent': 'task_class'}, {'text': 'classifies spam texts', 'intent': 'task_class'}, {'text': 'recognize face expressions ', 'intent': 'task_class'}, {'text': 'construct a model that recognizes if a woman is sad or happy', 'intent': 'task_class'}, {'text': 'make a model that tells if a drawing is a cat', 'intent': 'task_class'}, {'text': 'identify likely epicenters of outbreaks', 'intent': 'task_class'}, {'text': 'build a model that knows whether a medical procedure will succeed', 'intent': 'task_class'}, {'text': 'detect fraud', 'intent': 'task_class'}, {'text': 'automated detection of safe drinking water', 'intent': 'task_class'}, {'te

In [239]:
rsp = requests.post(WIT_API_HOST + '/utterances', headers=HEADERS, data=str(trainJson))
rsp.json()

{'error': 'Failed to validate utterance at index 0', 'code': 'bad-request'}

In [ ]:
#getQueries()

In [ ]:
#delQueries(getQueries())